In [341]:
import pandas as pd
import os
from utils import *
from ast import literal_eval

In [342]:
GAME_FOLDER = 'output/gpt-3_5-turbo-games/game_2024-03-09-14-44-25'

In [343]:
timestamp = GAME_FOLDER.split("_")[-1]
timestamp

'2024-03-09-14-44-25'

In [344]:
conversation_df = pd.read_csv(f'{GAME_FOLDER}/conversation.csv')
conversation_df.head()

,agent_name,visible_to,content,turn,timestamp,msg_type
0,Moderator,all,"This is round 1. For this round, the leader is...",1,1710024262014179000,NaN
1,Moderator,all,Now the leader choose your teammates and expla...,1,1710024262014179000,NaN
2,Moderator,Player1,Aim for a good team without evil player,1,1710024262014179000,role_tips
3,Moderator,Player1,You are percival and you need to help Merlin. ...,1,1710024262014179000,role_tips
4,Moderator,Player1,Intents are used to guide your gameplay and di...,1,1710024262014179000,intent


In [345]:
conversation_df['content'] = conversation_df['content'].astype(str)
conversation_df['content_len'] = conversation_df['content'].apply(lambda x: len(x))
conversation_df.head()

,agent_name,visible_to,content,turn,timestamp,msg_type,content_len
0,Moderator,all,"This is round 1. For this round, the leader is...",1,1710024262014179000,NaN,54
1,Moderator,all,Now the leader choose your teammates and expla...,1,1710024262014179000,NaN,632
2,Moderator,Player1,Aim for a good team without evil player,1,1710024262014179000,role_tips,39
3,Moderator,Player1,You are percival and you need to help Merlin. ...,1,1710024262014179000,role_tips,514
4,Moderator,Player1,Intents are used to guide your gameplay and di...,1,1710024262014179000,intent,1040


In [346]:
conversation_df['content_len'].describe()

count    1987.000000
mean      715.651233
std      1048.823040
min         2.000000
25%       190.000000
50%       281.000000
75%       618.000000
max      4070.000000
Name: content_len, dtype: float64

In [347]:
# print(conversation_df[conversation_df['content_len'] > 1500].sort_values('content_len', ascending=False)['content'].unique()[1])

In [348]:
conversation_df['agent_name'].unique()

array(['Moderator', 'Player1', 'Player1(Percival)', 'Player2(Morgana)',
       'Player2', 'Player3(Servant)', 'Player3', 'Player4(Assassin)',
       'Player4', 'Player5(Merlin)', 'Player5'], dtype=object)

In [349]:
conversation_df['name'] = conversation_df['agent_name'].apply(lambda x: x.split('(')[0])
conversation_df['name'].unique()

array(['Moderator', 'Player1', 'Player2', 'Player3', 'Player4', 'Player5'],
      dtype=object)

In [350]:
conversation_df['turn'].unique()

array([1, 2, 3, 4, 5])

In [351]:
conversation_df['round'] = conversation_df['turn']

In [352]:
impactful_intents = {
    'cast suspicion on innocent players': True, 
    'counter the team proposal because you think a player on the team is evil': True, 
    'express concerns about a player from a failed quest team and suggest to not include them in the current team': True, 
    'express confusion for failure of mission': True, 
    'express disagreement and vote disagree only if you are not in the proposed team': True, 
    'express you are a loyal player': True, 
    'pretend to be merlin by providing hints on who is evil': True, 
    'question a player about why they changed their statements': True, 
    'question a player on why they didnt vote for the last team': True, 
    'question the leader why they selected a particular player for the team': True, 
    'reveal identity by telling others you are servant to persuade other loyal players': True, 
    'share concerns about an evil player': True, 
    'strongly convince the team leader to choose you on team': True, 
    'support loyal players of the previous quest team if the mission failed': True, 
    'support your teammate by expressing that he is good': True
}

In [353]:
conversation_df[(conversation_df['round'] == 2) & (conversation_df['name'] == 'Player1')]['msg_type']

344        first_order
347        first_order
350        first_order
353        first_order
356             intent
357             intent
360    formulation_con
361    formulation_con
364       second_order
367             intent
368             intent
372           thinking
373         discussion
395        explanation
404            summary
527        first_order
530        first_order
533        first_order
536        first_order
539             intent
540             intent
543    formulation_con
544    formulation_con
547       second_order
550             intent
551             intent
555           thinking
556         discussion
609        explanation
622             action
623        explanation
624            answers
638            summary
Name: msg_type, dtype: object

In [354]:
conversation_df[conversation_df['content'] == "Most people don't agree with the team formation, we gonna move the leadership to next person and discuss again."].index

Int64Index([401, 852, 1298, 1749], dtype='int64')

In [355]:
secondary_round_start_indices = conversation_df[conversation_df['content'].str.startswith("Now the leader of this round is")].index.tolist()
secondary_round_start_indices

[417, 868, 1314, 1765]

In [356]:
next_round_start_indices = conversation_df[conversation_df['content'].str.startswith("This is round")].index[1:].tolist()
# appending end of game index
next_round_start_indices.append(len(conversation_df) - 1)
next_round_start_indices

[200, 651, 1097, 1548, 1986]

In [357]:
secondary_round_index_ranges = []

next_round_pointer = 0
for idx in secondary_round_start_indices:
    next_round_idx = next_round_start_indices[next_round_pointer]
    while next_round_idx < idx and next_round_pointer < len(next_round_start_indices) - 1:
        next_round_pointer += 1
        next_round_idx = next_round_start_indices[next_round_pointer]
    
    secondary_round_index_ranges.append((idx, next_round_idx - 1))

secondary_round_index_ranges

[(417, 650), (868, 1096), (1314, 1547), (1765, 1985)]

In [412]:
len(secondary_round_index_ranges)

4

In [358]:
def set_secondary_round(idx):
    for start, end in secondary_round_index_ranges:
        if start <= idx <= end:
            return True
    return False

In [359]:
conversation_df['index'] = conversation_df.index
conversation_df['is_secondary_round'] = conversation_df['index'].apply(lambda idx: set_secondary_round(idx))

conversation_df.head()

,agent_name,visible_to,content,turn,timestamp,msg_type,content_len,name,round,index,is_secondary_round
0,Moderator,all,"This is round 1. For this round, the leader is...",1,1710024262014179000,NaN,54,Moderator,1,0,False
1,Moderator,all,Now the leader choose your teammates and expla...,1,1710024262014179000,NaN,632,Moderator,1,1,False
2,Moderator,Player1,Aim for a good team without evil player,1,1710024262014179000,role_tips,39,Moderator,1,2,False
3,Moderator,Player1,You are percival and you need to help Merlin. ...,1,1710024262014179000,role_tips,514,Moderator,1,3,False
4,Moderator,Player1,Intents are used to guide your gameplay and di...,1,1710024262014179000,intent,1040,Moderator,1,4,False


In [360]:
conversation_df[(conversation_df['round'] == 2) & (conversation_df['name'] == 'Player1')  & (conversation_df['is_secondary_round'] == True)]['msg_type']

527        first_order
530        first_order
533        first_order
536        first_order
539             intent
540             intent
543    formulation_con
544    formulation_con
547       second_order
550             intent
551             intent
555           thinking
556         discussion
609        explanation
622             action
623        explanation
624            answers
638            summary
Name: msg_type, dtype: object

In [361]:
conversation_df[(conversation_df['round'] == 2) & (conversation_df['name'] == 'Player1')  & (conversation_df['is_secondary_round'] == False)]['msg_type']

344        first_order
347        first_order
350        first_order
353        first_order
356             intent
357             intent
360    formulation_con
361    formulation_con
364       second_order
367             intent
368             intent
372           thinking
373         discussion
395        explanation
404            summary
Name: msg_type, dtype: object

In [362]:
team_proposal_mod_prompt_indices = conversation_df[conversation_df['content'].str.contains("do you want to change your team proposal?")].index.tolist()
team_proposal_mod_prompt_indices

[146, 374, 592, 825, 1043, 1271, 1489, 1722, 1940]

In [363]:
def set_team_proposal_change_message_type(idx):
    found = False
    idx_discussion = idx
    while conversation_df.iloc[idx_discussion]['msg_type'] != 'discussion':
        idx_discussion += 1
    conversation_df.at[idx_discussion, 'msg_type'] = 'reconsider_team'

In [364]:
conversation_df.iloc[150]['msg_type']

'discussion'

In [365]:
for idx in team_proposal_mod_prompt_indices:
    set_team_proposal_change_message_type(idx)

In [366]:
conversation_df.iloc[150]['msg_type']

'reconsider_team'

In [367]:
player_discussion_end_indices = conversation_df[conversation_df['msg_type'] == 'discussion'].index.tolist()
len(player_discussion_end_indices)

45

In [368]:
len([x for x in player_discussion_end_indices if x < 200])

5

In [369]:
next_round_start_indices

[200, 651, 1097, 1548, 1986]

In [370]:
def check_is_start_of_player_discussion(content: str):
    if "respond with your views on the team proposal" in content:
        return True
    if "Now the leader choose your teammates" in content:
        return True
    if "This is round" in content:
        return True
    return False

In [371]:
player_discussion_start_indices = conversation_df[conversation_df['content'].apply(lambda x: check_is_start_of_player_discussion(x))].index.tolist()[1:]
len(player_discussion_start_indices)

45

In [372]:
player_discussion_start_indices[:10]

[1, 24, 50, 79, 111, 200, 234, 269, 304, 339]

In [373]:
player_discussion_range_indices = []
for start, end in zip(player_discussion_start_indices, player_discussion_end_indices):
    player_discussion_range_indices.append((start, end))

In [374]:
player_discussion_range_indices[:10]

[(1, 23),
 (24, 49),
 (50, 78),
 (79, 110),
 (111, 145),
 (200, 233),
 (234, 268),
 (269, 303),
 (304, 338),
 (339, 373)]

In [380]:
def filter_out_non_impactful_intents(intents, all=False):
    final_intents = []
    non_impactful_present = False
    for intent in intents:
        norm_intent = remove_non_alphabets(intent)
        if norm_intent in impactful_intents:
            final_intents.append(intent)
        elif all: non_impactful_present = True
    if all and non_impactful_present:
        return []
    return final_intents

In [420]:
def process_player_discussion(start_idx, end_idx, df):
    # print(conversation_df.iloc[start_idx: end_idx+1]['msg_type'])
    intent_count = 0
    cur_idx = start_idx
    msg_type = ''
    while cur_idx <= end_idx:
        msg_type = df.iloc[cur_idx]['msg_type']
        if msg_type == 'intent':
            intent_count += 1
            if intent_count == 2:
                break
        cur_idx += 1

    if msg_type != 'intent':
        return
    orig_intent_idx = cur_idx
    orig_selected_intents = literal_eval(df.iloc[orig_intent_idx]['content'])
    # print(f"{orig_selected_intents=}")

    intent_mod_prompt_idx = -1
    intent_count = 0
    while cur_idx <= end_idx:
        msg_type = df.iloc[cur_idx]['msg_type']
        if msg_type == 'intent_modification':
            intent_mod_prompt_idx = cur_idx
        if msg_type == 'intent' and intent_mod_prompt_idx != -1:
            break
        cur_idx += 1
    
    if msg_type != 'intent':
        return
    mod_intent_idx = cur_idx
    modified_intents = literal_eval(df.iloc[mod_intent_idx]['content'])
    # print(f"{modified_intents=}")

    retained_orig_intents = filter_out_non_impactful_intents(orig_selected_intents, all=True)
    retained_mod_intents = filter_out_non_impactful_intents(modified_intents, all=True)

    annotation_data = {
        "round": df.loc[orig_intent_idx]['round'],
        "is_secondary_round": df.loc[orig_intent_idx]['is_secondary_round'],
        "player": df.loc[orig_intent_idx]['name'],
        "updated_orig_intents": (orig_intent_idx, retained_orig_intents),
        "updated_mod_intents": (mod_intent_idx, retained_mod_intents),
        "was_intent_modified": len(modified_intents) > 0
    }

    retained_annotation_data = {}
    if len(retained_orig_intents) > 0 or len(retained_mod_intents) > 0:
        retained_annotation_data = annotation_data

    # print(f"{retained_orig_intents=}\n{retained_mod_intents=}")

    # print(intent_mod_prompt_idx)

    to_drop_indices = []

    if len(retained_orig_intents) == 0 and len(retained_mod_intents) == 0:
        to_drop_indices.extend(df.index[start_idx + 1: end_idx].tolist())
        # df.drop(index=df.index[start_idx + 1: end_idx], inplace=True)
    elif len(retained_orig_intents) == 0:
        to_drop_indices.extend(df.index[start_idx + 1: intent_mod_prompt_idx].tolist())
        # df.drop(index=df.index[start_idx + 1: intent_mod_prompt_idx], inplace=True)
    elif len(modified_intents) != 0 and len(retained_mod_intents) == 0:
        to_drop_indices.extend(df.index[intent_mod_prompt_idx: end_idx].tolist())
        # df.drop(index=df.index[intent_mod_prompt_idx: end_idx], inplace=True)

    # print(df.loc[start_idx: end_idx]['msg_type'])
    return annotation_data, to_drop_indices, retained_annotation_data
        

In [421]:
conversation_df_filtered = conversation_df.copy()

In [426]:
final_drop_indices = []
annotation_file_data = []
retained_annotation_file_data = []

for start_idx, end_idx in player_discussion_range_indices:
    annotation_data, to_drop_indices, retained_annotation_data = process_player_discussion(start_idx, end_idx, conversation_df_filtered)
    final_drop_indices.extend(to_drop_indices)
    annotation_file_data.append(annotation_data)
    if len(retained_annotation_data) != 0:
        retained_annotation_file_data.append(retained_annotation_data)

In [427]:
len(retained_annotation_file_data)

10

In [424]:
retained_annotation_file_data

[{'round': 1,
  'is_secondary_round': False,
  'player': 'Player1',
  'updated_orig_intents': (6,
   ['Pretend to be Merlin by providing hints on who is evil.']),
  'updated_mod_intents': (17, []),
  'was_intent_modified': False},
 {'round': 2,
  'is_secondary_round': False,
  'player': 'Player3',
  'updated_orig_intents': (251,
   ['Counter the team proposal because you think a player on the team is evil',
    'Question the leader why they selected a particular player for the team',
    'Express concerns about a player from a failed quest team and suggest to not include them in the current team']),
  'updated_mod_intents': (262,
   ['Counter the team proposal because you think a player on the team is evil',
    'Question the leader why they selected a particular player for the team',
    'Express concerns about a player from a failed quest team and suggest to not include them in the current team']),
  'was_intent_modified': True},
 {'round': 2,
  'is_secondary_round': True,
  'player'

In [384]:
len(final_drop_indices)

1131

In [385]:
len(conversation_df_filtered)

1987

In [386]:
len(annotation_file_data)

45

In [391]:
intent_count = 0
for ann in annotation_file_data:
    intent_count += len(ann['updated_orig_intents'][1])
    intent_count += len(ann['updated_mod_intents'][1])

print(intent_count)

31


In [415]:
annotation_file_data[10]

{'round': 2,
 'is_secondary_round': True,
 'player': 'Player3',
 'updated_orig_intents': (434, []),
 'updated_mod_intents': (445, []),
 'was_intent_modified': False}

In [436]:
for ann in annotation_file_data:
    orig_intents_idx, updated_orig_intents = ann['updated_orig_intents']
    mod_intents_idx, updated_mod_intents = ann['updated_mod_intents']
    
    conversation_df_filtered.at[orig_intents_idx, 'content'] = updated_orig_intents
    conversation_df_filtered.at[mod_intents_idx, 'content'] = updated_mod_intents

In [437]:
conversation_df.iloc[434]['content']

"['Strongly convince the team leader to choose you on the team.', 'Reveal identity by telling others you are Servant to persuade other loyal players.', 'Pretend to be Merlin by providing hints on who is evil.']"

In [438]:
conversation_df_filtered.iloc[434]['content']

[]

In [439]:
conversation_df_filtered.drop(index=final_drop_indices, inplace=True)

In [440]:
len(conversation_df_filtered)

856

In [441]:
conversation_df_filtered.columns

Index(['agent_name', 'visible_to', 'content', 'turn', 'timestamp', 'msg_type',
       'content_len', 'name', 'round', 'index', 'is_secondary_round'],
      dtype='object')

In [442]:
conversation_df_filtered.reset_index(drop=True, inplace=True)
conversation_df_filtered['index'] = conversation_df_filtered.index
conversation_df_filtered = conversation_df_filtered[['index', 'name', 'round', 'is_secondary_round', 'agent_name', 'visible_to', 'msg_type', 'content', 'turn', 'timestamp']]
conversation_df_filtered.head()

,index,name,round,is_secondary_round,agent_name,visible_to,msg_type,content,turn,timestamp
0,0,Moderator,1,False,Moderator,all,NaN,"This is round 1. For this round, the leader is...",1,1710024262014179000
1,1,Moderator,1,False,Moderator,all,NaN,Now the leader choose your teammates and expla...,1,1710024262014179000
2,2,Moderator,1,False,Moderator,Player1,role_tips,Aim for a good team without evil player,1,1710024262014179000
3,3,Moderator,1,False,Moderator,Player1,role_tips,You are percival and you need to help Merlin. ...,1,1710024262014179000
4,4,Moderator,1,False,Moderator,Player1,intent,Intents are used to guide your gameplay and di...,1,1710024262014179000


In [443]:
conversation_df_filtered.to_csv(f'{GAME_FOLDER}/conversation-data-oi-{timestamp}.csv', index=False)

In [335]:
annotation_file_columns = ["player", "round", "is_secondary_round", "intent", "intent_selection", "intent_fol_think_1", "intent_fol_speech_1", "intent_mod", "intent_selection_mod", "intent_fol_think_2", "intent_fol_speech_2", "comments"]
annotation_file_df = pd.DataFrame(columns=annotation_file_columns)
annotation_file_df.head()

,player,round,is_secondary_round,intent,intent_selection,intent_fol_think_1,intent_fol_speech_1,intent_mod,intent_selection_mod,intent_fol_think_2,intent_fol_speech_2,comments


In [336]:
annotation_file_data[10]

{'round': 2,
 'is_secondary_round': True,
 'player': 'Player3',
 'updated_orig_intents': (434,
  ['Reveal identity by telling others you are Servant to persuade other loyal players.',
   'Pretend to be Merlin by providing hints on who is evil.']),
 'updated_mod_intents': (445, [])}

In [428]:
retained_annotation_file_data[9]

{'round': 5,
 'is_secondary_round': True,
 'player': 'Player3',
 'updated_orig_intents': (1922,
  ['Express concerns about a player from a failed quest team and suggest to not include them in the current team.',
   'Counter the team proposal because you think a player on the team is evil.']),
 'updated_mod_intents': (1933, []),
 'was_intent_modified': False}

In [433]:
def get_row_data(ann):
    row_data = {col:None for col in annotation_file_columns}
    row_data["player"] = ann["player"]
    row_data["round"] = ann["round"]
    row_data["is_secondary_round"] = ann["is_secondary_round"]
    row_data["intent"] = ann["updated_orig_intents"][1]
    if ann["was_intent_modified"]:
        row_data["intent_mod"] = ann["updated_mod_intents"][1]
    else:
        row_data["intent_mod"] = ann["updated_orig_intents"][1]

    return row_data
    

In [434]:
annotation_file_df = pd.concat([pd.DataFrame([get_row_data(ann)], columns=annotation_file_columns) for ann in retained_annotation_file_data],
          ignore_index=True)

In [435]:
annotation_file_df.head(10)

,player,round,is_secondary_round,intent,intent_selection,intent_fol_think_1,intent_fol_speech_1,intent_mod,intent_selection_mod,intent_fol_think_2,intent_fol_speech_2,comments
0,Player1,1,False,[Pretend to be Merlin by providing hints on wh...,None,None,None,[Pretend to be Merlin by providing hints on wh...,None,None,None,None
1,Player3,2,False,[Counter the team proposal because you think a...,None,None,None,[Counter the team proposal because you think a...,None,None,None,None
2,Player1,2,True,[Express concerns about a player from a failed...,None,None,None,[Express concerns about a player from a failed...,None,None,None,None
3,Player3,3,False,[Express concerns about a player from a failed...,None,None,None,[Express concerns about a player from a failed...,None,None,None,None
4,Player2,3,True,[Express concerns about a player from a failed...,None,None,None,[Express concerns about a player from a failed...,None,None,None,None
5,Player3,3,True,[Express concerns about a player from a failed...,None,None,None,[Express concerns about a player from a failed...,None,None,None,None
6,Player1,4,False,[Pretend to be Merlin by providing hints on wh...,None,None,None,[Pretend to be Merlin by providing hints on wh...,None,None,None,None
7,Player2,4,False,[Question the leader why they selected a parti...,None,None,None,[Counter the team proposal because you think a...,None,None,None,None
8,Player3,5,False,[Reveal identity by telling others you are Ser...,None,None,None,[Reveal identity by telling others you are Ser...,None,None,None,None
9,Player3,5,True,[Express concerns about a player from a failed...,None,None,None,[Express concerns about a player from a failed...,None,None,None,None


In [340]:
annotation_file_df.to_csv(f'{GAME_FOLDER}/annotation-{timestamp}.csv', index=False)